In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset
from datetime import datetime
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from timeit import default_timer as timer



import math

In [2]:

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)

        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.src_tok_emb = nn.Linear(src_vocab_size, emb_size)

        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)
        self.bn1=nn.BatchNorm1d(BATCH_SIZE)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                memory_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         src_emb = self.positional_encoding(src)
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg).squeeze())
#         outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
#                                 src_padding_mask, tgt_padding_mask, memory_key_padding_mask)


        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask,memory_mask=memory_mask)
        outs = self.bn1(outs)
        return self.generator(outs)

#     def encode(self, src: Tensor, src_mask: Tensor):
#         return self.transformer.encoder(self.positional_encoding(
#                             self.src_tok_emb(src)), src_mask)

#     def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
#         return self.transformer.decoder(self.positional_encoding(
#                           self.tgt_tok_emb(tgt)), memory,
#                           tgt_mask)

In [3]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask
def generate_square_subsequent_mask2(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1)
    mask = mask.float().masked_fill(mask == 1, float('-inf')).masked_fill(mask == 0, float(0.0))
    return mask
def generate_square_subsequent_mask3(sz):
    mask = (torch.triu(torch.ones((sz, sz-1), device=DEVICE)) == 1).transpose(0,1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)
    src_mask = generate_square_subsequent_mask(src_seq_len)
    mmr_mask = generate_square_subsequent_mask3(src_seq_len)
    return src_mask, tgt_mask ,mmr_mask

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def batchify(data, bsz):
#     # 데이터셋을 bsz 파트들로 나눕니다.
#     nbatch = data.size(0) // bsz
#     # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
#     data = data.narrow(0, 0, nbatch * bsz)
#     # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
#     data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
#     return data.to(device)

def batchify(data, bsz,bptt):
    # 데이터셋을 bsz 파트들로 나눕니다.
    if(data.size(0)%(bsz*bptt)!=0):
        data=data.view(-1,bptt,data.size(1)).transpose(0,1).contiguous()
        return data.to(device)
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)


bptt = 39
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source)  - i)
    data = source[i:i+seq_len]
    if(seq_len!=bptt*bs):
        print(seq_len)
    target = source[i:i+seq_len].reshape(-1)
    return data, target

In [5]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    conf00=0
    conf01=0
    conf02=0
    conf10=0
    conf11=0
    conf12=0
    conf20=0
    conf21=0
    conf22=0
    
    
    
    i=0
    while(i<Xtest_data.size(0)):
#     for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)

        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE,bptt)
        tgt=batchify(targets,BATCH_SIZE,bptt)
        src_input=src[:]
        tgt_input = tgt[:-1]


#         src_mask, tgt_mask = create_mask(src_input, tgt_input)
#         logits = model(src_input, tgt_input, src_mask, tgt_mask)
        src_mask, tgt_mask,mmr_mask = create_mask(src_input, tgt_input)
        logits = model(src_input, tgt_input, src_mask, tgt_mask,mmr_mask)
        
        optimizer.zero_grad()

        tgt_out = tgt[1:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
        if(loss.isnan()):
            print('NNNNN')
            print(loss)
        _,predicted=torch.max(logits,-1)
        correct += (tgt_out.squeeze() == predicted).sum().item()
        total+=len(predicted)*BATCH_SIZE
        tot0+=(0== tgt_out.squeeze()).sum().item()
        tot1+=(1== tgt_out.squeeze()).sum().item()
        tot2+=(2== tgt_out.squeeze()).sum().item()
        correct0+=((0== predicted) &(0==tgt_out.squeeze())).sum().item()
        correct1+=((1== predicted)&(1==tgt_out.squeeze())).sum().item()
        correct2+=((2== predicted)&(2==tgt_out.squeeze())).sum().item()
        
        conf00+=((0== predicted) &(0==tgt_out.squeeze())).sum().item()
        conf01+=((0== predicted)&(1==tgt_out.squeeze())).sum().item()
        conf02+=((0== predicted)&(2==tgt_out.squeeze())).sum().item()
        conf10+=((1== predicted) &(0==tgt_out.squeeze())).sum().item()
        conf11+=((1== predicted)&(1==tgt_out.squeeze())).sum().item()
        conf12+=((1== predicted)&(2==tgt_out.squeeze())).sum().item()
        conf20+=((2== predicted) &(0==tgt_out.squeeze())).sum().item()
        conf21+=((2== predicted)&(1==tgt_out.squeeze())).sum().item()
        conf22+=((2== predicted)&(2==tgt_out.squeeze())).sum().item()

        
        
        
        i+=targets.size()[0]
    tp0=conf00
    fp0=conf01+conf02
    fn0=conf10+conf20
    if(tp0+fp0==0):
        prec0=0
    else:
        prec0=tp0/(tp0+fp0)
    if(tp0+fn0==0):
        reca0=0
    else:
        reca0=tp0/(tp0+fn0)
    
    tp1=conf11
    fp1=conf10+conf12
    fn1=conf01+conf21
    
    if(tp1+fp1==0):
        prec1=0
    else:
        prec1=tp1/(tp1+fp1)
    if(tp1+fn1==0):
        reca1=0
    else:
        reca1=tp1/(tp1+fn1)
    
    tp2=conf22
    fp2=conf20+conf21
    fn2=conf02+conf12
    
    if(tp2+fp2==0):
        prec2=0
    else:
        prec2=tp2/(tp2+fp2)
    if(tp2+fn2==0):
        reca2=0
    else:
        reca2=tp2/(tp2+fn2)
        
    prec=(prec0+prec1+prec2)/3
    reca=(reca0+reca1+reca2)/3
    f1sc=2*(prec*reca)/(prec+reca)
    print(total)
    print(correct)
    print("Acc:",correct/total)            
    print("Prec",prec)
    print("Recall",reca)
    print("F1",f1sc) 
    return losses / Xtrain_data.size(0),[conf00,conf01,conf02,conf10,conf11,conf12,conf20,conf21,conf22]

In [6]:
def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    conf00=0
    conf01=0
    conf02=0
    conf10=0
    conf11=0
    conf12=0
    conf20=0
    conf21=0
    conf22=0
    with torch.no_grad():
        i=0
        while(i<Xtest_data.size(0)):
#         for i in range(0, Xtest_data.size(0) - 1, bptt):

            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE,bptt)
            tgt=batchify(targets,BATCH_SIZE,bptt)

            src_input=src[:]
            
            tgt_input = tgt[:-1]
#             print(src_input.size())
#             print(tgt_input.size())
            src_mask, tgt_mask,mmr_mask = create_mask(src_input, tgt_input)

#             logits = model(src_input, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
            logits = model(src_input, tgt_input, src_mask, tgt_mask,mmr_mask)

            
            tgt_out = tgt[1:]

            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            
            
            if(loss.isnan()):
                print(src,tgt_input)
                break
            losses += loss.item()
            _,predicted=torch.max(logits,-1)
            
            correct += (tgt_out.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt_out.squeeze()).sum().item()
            tot1+=(1== tgt_out.squeeze()).sum().item()
            tot2+=(2== tgt_out.squeeze()).sum().item()

            conf00+=((0== predicted) &(0==tgt_out.squeeze())).sum().item()
            conf01+=((0== predicted)&(1==tgt_out.squeeze())).sum().item()
            conf02+=((0== predicted)&(2==tgt_out.squeeze())).sum().item()
            conf10+=((1== predicted) &(0==tgt_out.squeeze())).sum().item()
            conf11+=((1== predicted)&(1==tgt_out.squeeze())).sum().item()
            conf12+=((1== predicted)&(2==tgt_out.squeeze())).sum().item()
            conf20+=((2== predicted) &(0==tgt_out.squeeze())).sum().item()
            conf21+=((2== predicted)&(1==tgt_out.squeeze())).sum().item()
            conf22+=((2== predicted)&(2==tgt_out.squeeze())).sum().item()
            

            
            i+=targets.size()[0]
    tp0=conf00
    fp0=conf01+conf02
    fn0=conf10+conf20
    if(tp0+fp0==0):
        prec0=0
    else:
        prec0=tp0/(tp0+fp0)
    if(tp0+fn0==0):
        reca0=0
    else:
        reca0=tp0/(tp0+fn0)
    
    tp1=conf11
    fp1=conf10+conf12
    fn1=conf01+conf21
    
    if(tp1+fp1==0):
        prec1=0
    else:
        prec1=tp1/(tp1+fp1)
    if(tp1+fn1==0):
        reca1=0
    else:
        reca1=tp1/(tp1+fn1)
    
    tp2=conf22
    fp2=conf20+conf21
    fn2=conf02+conf12
    
    if(tp2+fp2==0):
        prec2=0
    else:
        prec2=tp2/(tp2+fp2)
    if(tp2+fn2==0):
        reca2=0
    else:
        reca2=tp2/(tp2+fn2)
    
    prec=(prec0+prec1+prec2)/3
    reca=(reca0+reca1+reca2)/3
    f1sc=2*(prec*reca)/(prec+reca)
    print(total)
    print(correct)
    print("Acc:",correct/total)            
    print("Prec",prec)
    print("Recall",reca)
    print("F1",f1sc)
    return losses / Xtest_data.size(0),correct/total,prec,reca,f1sc,[conf00,conf01,conf02,conf10,conf11,conf12,conf20,conf21,conf22]

In [7]:
mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]

mbrnlist=mbrnlist1+mbrnlist2
mbrnlist=set(mbrnlist)
mbrnlist=list(mbrnlist)

# load array
MBR_NO,BRN_NO=mbrnlist[2]
featnorm=True

In [8]:

for MBR_NO,BRN_NO in mbrnlist:
    if featnorm==True:
        Data_train = load('DataMid_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_train.npy',allow_pickle=True)
        Data_test =  load('DataMid_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_test.npy',allow_pickle=True)
    else:
        Data_train = load('DataMid_'+str(MBR_NO)+'_'+str(BRN_NO)+'train.npy',allow_pickle=True)
        Data_test =  load('DataMid_'+str(MBR_NO)+'_'+str(BRN_NO)+'test.npy',allow_pickle=True)
    
#     if featnorm==True:
#         Data_train = load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_train.npy',allow_pickle=True)
#         Data_test =  load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_test.npy',allow_pickle=True)
#     else:
#         Data_train = load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'train.npy',allow_pickle=True)
#         Data_test =  load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'test.npy',allow_pickle=True)



    Xdata=[]
    Ydata=[]
    Xtrain_data=[]
    Ytrain_data=[]
    Xtest_data=[]
    Ytest_data=[]



    for idx in range(len(Data_train)//39):
        if(np.isinf(Data_train[39*idx:39*(idx+1)][:,:].tolist()).any()):
            print(np.isinf(Data_train[39*idx:39*(idx+1)][:,:].tolist()).any())
            continue
        Xtrain_data.append(Data_train[39*idx:39*(idx+1)][:,:].tolist())
        Ytrain_data.append(Data_train[39*idx:39*(idx+1)][:,-1].tolist())
    for idx in range(len(Data_test)//39):
        if(np.isinf(Data_test[39*idx:39*(idx+1)][:,:].tolist()).any()):
            print(np.isinf(Data_test[39*idx:39*(idx+1)][:,:].tolist()).any())
            continue
        Xtest_data.append(Data_test[39*idx:39*(idx+1)][:,:].tolist())
        Ytest_data.append(Data_test[39*idx:39*(idx+1)][:,-1].tolist())

    Xtrain_data=np.vstack(Xtrain_data)
    Ytrain_data=np.vstack(Ytrain_data)
    Xtrain_data=torch.FloatTensor(Xtrain_data)
    Ytrain_data=torch.IntTensor(Ytrain_data)
    Ytrain_data=Ytrain_data.view(-1)
    Ytrain_data=torch.cat(((Ytrain_data[1:]-Ytrain_data[:-1]),torch.tensor(0).unsqueeze(-1)))
    Ytrain_data[::39]=0
    
    Xtest_data=np.vstack(Xtest_data)
    Ytest_data=np.vstack(Ytest_data)
    Xtest_data=torch.FloatTensor(Xtest_data)
    Ytest_data=torch.IntTensor(Ytest_data)
    Ytest_data=Ytest_data.view(-1)
    Ytest_data=torch.cat(((Ytest_data[1:]-Ytest_data[:-1]),torch.tensor(0).unsqueeze(-1)))
    Ytest_data[::39]=0
    
    
    Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
    Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()
    Ytrain_data=Ytrain_data.T
    Ytest_data=Ytest_data.T

    torch.manual_seed(0)

    SRC_VOCAB_SIZE = Xtrain_data.shape[1]
    TGT_VOCAB_SIZE = 3
    EMB_SIZE = 512
    NHEAD = 8
    FFN_HID_DIM = 512
    BATCH_SIZE = 10
    NUM_ENCODER_LAYERS = 3
    NUM_DECODER_LAYERS = 3

    transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                     NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    transformer = transformer.to(DEVICE)

    loss_fn = torch.nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(transformer.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

    NUM_EPOCHS = 300
    best_val_loss=100000000
    for epoch in range(1, NUM_EPOCHS+1):
        start_time = timer()
        train_loss,_ = train_epoch(transformer, optimizer)
        end_time = timer()
        val_loss,acc,prec,reca,f1sc,confusion = evaluate(transformer)
        print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_confusion=confusion
            best_acc=acc
            best_prec=prec
            best_reca=reca
            best_f1sc=f1sc
            best_model = transformer

    now = datetime.now()
    now.strftime("%m/%d/%Y, %H:%M:%S")

    date_time = now.strftime("%m_%d_%Y")

    PATH='best_model_Trans_seq_'+date_time+'_'+str(MBR_NO)+'_'+str(BRN_NO)+'Mid'
    if featnorm==True:
        torch.save(best_model.state_dict(), PATH+'norm')
    else:
        torch.save(best_model.state_dict(), PATH)
    if featnorm==True:
        file_name='result_Trans_'+date_time+'_norm.txt'
    else:
        file_name='result_Trans_'+date_time+'.txt'
    text_to_append=PATH+'\t'+"Acc:"+str(best_acc)+'\t'+"prec:"+str(best_prec)+'\t'+"recall:"+str(best_reca)+'\t'+"f1sc:"+str(best_f1sc)
    print(text_to_append)
    with open(file_name, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(text_to_append)
    break

7600
2954
Acc: 0.3886842105263158
Prec 0.3593729010953221
Recall 0.3544226605657836
F1 0.3568806156346505
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 1, Train loss: 0.001, Val loss: 0.003, Epoch time = 1.112s
7600
2832
Acc: 0.3726315789473684
Prec 0.3103898899744466
Recall 0.31707390242112826
F1 0.31369629571937835
7600
2849
Acc: 0.3748684210526316
Prec 0.1249561403508772
Recall 0.3333333333333333
F1 0.1817717803936581
Epoch: 2, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.700s
7600
2786
Acc: 0.36657894736842106
Prec 0.3103794275768742
Recall 0.3155146708391807
F1 0.3129259827660928
7600
2849
Acc: 0.3748684210526316
Prec 0.1249561403508772
Recall 0.3333333333333333
F1 0.1817717803936581
Epoch: 3, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.679s
7600
2887
Acc: 0.3798684210526316
Prec 0.3126452104191362
Recall 0.32155263265846973
F1 0.3170363683688294
7600
2849
Acc: 0.3748684210526316
Prec 0.1249561403508

7600
3174
Acc: 0.4176315789473684
Prec 0.2779456053656533
Recall 0.3422332955290723
F1 0.3067574223014639
7600
2849
Acc: 0.3748684210526316
Prec 0.1249561403508772
Recall 0.3333333333333333
F1 0.1817717803936581
Epoch: 31, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.662s
7600
3138
Acc: 0.41289473684210526
Prec 0.2750789645866562
Recall 0.338542904989312
F1 0.30352905067406305
7600
2849
Acc: 0.3748684210526316
Prec 0.1249561403508772
Recall 0.3333333333333333
F1 0.1817717803936581
Epoch: 32, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.657s
7600
3120
Acc: 0.4105263157894737
Prec 0.2737574536447754
Recall 0.33620148208504
F1 0.30178314065381184
7600
2849
Acc: 0.3748684210526316
Prec 0.1249561403508772
Recall 0.3333333333333333
F1 0.1817717803936581
Epoch: 33, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.667s
7600
3098
Acc: 0.4076315789473684
Prec 0.27184016598442323
Recall 0.33384909832940607
F1 0.29967047346110165
7600
2849
Acc: 0.3748684210526316
Prec 0.1249561403508

7600
3165
Acc: 0.4164473684210526
Prec 0.2634975374378988
Recall 0.33121482852299644
F1 0.29350084738094756
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 61, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.647s
7600
3180
Acc: 0.41842105263157897
Prec 0.26415987158367615
Recall 0.3327032623159133
F1 0.29449582678908437
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 62, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.669s
7600
3166
Acc: 0.41657894736842105
Prec 0.25957969519308194
Recall 0.3305382094744674
F1 0.2907927618748881
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 63, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.630s
7600
3178
Acc: 0.4181578947368421
Prec 0.2651024772850686
Recall 0.3328395612143442
F1 0.2951342656483403
7600
3339
Acc: 0.43934210526315787
Prec 0.14

7600
3194
Acc: 0.42026315789473684
Prec 0.2658217790554213
Recall 0.33348596674726805
F1 0.2958340972284429
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 90, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.653s
7600
3191
Acc: 0.41986842105263156
Prec 0.26520771482429134
Recall 0.33340094897485256
F1 0.29542006036698215
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 91, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.684s
7600
3176
Acc: 0.41789473684210526
Prec 0.2615225305820168
Recall 0.3318179956325278
F1 0.29250616830138265
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 92, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.675s
7600
3175
Acc: 0.41776315789473684
Prec 0.2620260755283676
Recall 0.33166756862373736
F1 0.2927622764452171
7600
3339
Acc: 0.43934210526315787
Prec 0

7600
3158
Acc: 0.4155263157894737
Prec 0.25972277791401716
Recall 0.3301382095741115
F1 0.29072752633220683
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 119, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.693s
7600
3206
Acc: 0.4218421052631579
Prec 0.2639912185257723
Recall 0.33295173200490663
F1 0.2944882199014239
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 120, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.696s
7600
3161
Acc: 0.41592105263157897
Prec 0.25905051118534267
Recall 0.32996329858565515
F1 0.2902382244798247
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 121, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.656s
7600
3156
Acc: 0.41526315789473683
Prec 0.2593699210217783
Recall 0.3301209142047545
F1 0.29049963232090137
7600
3339
Acc: 0.43934210526315787
Prec 

...

7600
3169
Acc: 0.41697368421052633
Prec 0.25875895228040346
Recall 0.33028059388027914
F1 0.290177666297798
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 177, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.692s
7600
3166
Acc: 0.41657894736842105
Prec 0.2595122738520191
Recall 0.3304791347560874
F1 0.29072759524933084
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 178, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.701s
7600
3194
Acc: 0.42026315789473684
Prec 0.2626418087077207
Recall 0.33288340461979243
F1 0.2936201441064565
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 179, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.724s
7600
3172
Acc: 0.41736842105263156
Prec 0.2615861081821747
Recall 0.33144077152720997
F1 0.292399230062516
7600
3339
Acc: 0.43934210526315787
Prec 0

7600
3189
Acc: 0.41960526315789476
Prec 0.26216860974366746
Recall 0.332674756984936
F1 0.2932431743005236
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 206, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.669s
7600
3173
Acc: 0.4175
Prec 0.2644559664054968
Recall 0.3316857180854083
F1 0.29427993177187745
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 207, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.684s
7600
3161
Acc: 0.41592105263157897
Prec 0.2583808756795783
Recall 0.3300932629660911
F1 0.2898675769759269
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 208, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.685s
7600
3214
Acc: 0.42289473684210527
Prec 0.26082836869127196
Recall 0.3336589204408384
F1 0.2927824143891325
7600
3339
Acc: 0.43934210526315787
Prec 0.1464473684210

7600
3198
Acc: 0.42078947368421055
Prec 0.2619598156923473
Recall 0.3328589206401265
F1 0.29318397749227326
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 236, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.693s
7600
3181
Acc: 0.41855263157894734
Prec 0.2602305202991726
Recall 0.33163675012607646
F1 0.2916262086046038
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 237, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.654s
7600
3194
Acc: 0.42026315789473684
Prec 0.26334029506289397
Recall 0.3334268920288881
F1 0.2942679759478358
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 238, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.639s
7600
3154
Acc: 0.415
Prec 0.2632573270780509
Recall 0.3310960741041808
F1 0.2933051861109083
7600
3339
Acc: 0.43934210526315787
Prec 0.1464473684210

7600
3206
Acc: 0.4218421052631579
Prec 0.26926799395050627
Recall 0.3354092402895131
F1 0.29872126209193806
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 265, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.746s
7600
3216
Acc: 0.4231578947368421
Prec 0.24216216216216213
Recall 0.33249472144259606
F1 0.28022857795408884
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 266, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.655s
7600
3209
Acc: 0.42223684210526313
Prec 0.2695216599350598
Recall 0.33544699828722463
F1 0.2988922833268047
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 267, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.712s
7600
3191
Acc: 0.41986842105263156
Prec 0.2657064610390872
Recall 0.33357817312999244
F1 0.29579892694945165
7600
3339
Acc: 0.43934210526315787
Pre

7600
3158
Acc: 0.4155263157894737
Prec 0.26169664575685814
Recall 0.3306226222648272
F1 0.29214930504294967
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 294, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.671s
7600
3207
Acc: 0.42197368421052633
Prec 0.2704416065856867
Recall 0.3352642937064037
F1 0.29938428592837457
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 295, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.699s
7600
3200
Acc: 0.42105263157894735
Prec 0.2710485293865735
Recall 0.33568817260436984
F1 0.2999251082662834
7600
3339
Acc: 0.43934210526315787
Prec 0.14644736842105263
Recall 0.3333333333333333
F1 0.2034920925130268
Epoch: 296, Train loss: 0.000, Val loss: 0.003, Epoch time = 0.708s
7600
3204
Acc: 0.42157894736842105
Prec 0.2630042675468816
Recall 0.3329816964102536
F1 0.2938847975191426
7600
3339
Acc: 0.43934210526315787
Prec 0

In [9]:
Ytrain_data.size()

torch.Size([74100])

In [10]:
Ytrain_data

tensor([1, 0, 2,  ..., 1, 1, 1])

In [11]:
(Ytrain_data==0).sum()/Ytrain_data.size()[0]

tensor(0.4167)

In [12]:
(Ytrain_data==1).sum()/Ytrain_data.size()[0]

tensor(0.2212)

In [13]:
int((Ytrain_data==2).sum())/int(Ytrain_data.size()[0])

0.36214574898785423

In [14]:
Data_test[:,-1][:100]

array([43225.0, 43775.0, 43875.0, 43675.0, 43575.0, 43575.0, 43525.0,
       43275.0, 43225.0, 43625.0, 43325.0, 43275.0, 43275.0, 43275.0,
       43125.0, 43225.0, 43025.0, 43125.0, 43375.0, 43575.0, 43525.0,
       43725.0, 43825.0, 44075.0, 43825.0, 43850.0, 44125.0, 44175.0,
       43875.0, 43925.0, 43725.0, 43675.0, 43525.0, 43475.0, 43725.0,
       43625.0, 43375.0, 42975.0, 42925.0, 43425.0, 43125.0, 43475.0,
       43825.0, 43525.0, 43575.0, 43525.0, 44975.0, 44825.0, 45175.0,
       44775.0, 45125.0, 45125.0, 45025.0, 44975.0, 46475.0, 46325.0,
       45825.0, 45775.0, 45425.0, 45675.0, 45600.0, 45125.0, 44825.0,
       44675.0, 44675.0, 44625.0, 44875.0, 45025.0, 45150.0, 44925.0,
       45225.0, 45075.0, 44675.0, 44525.0, 44925.0, 44425.0, 44525.0,
       44675.0, 147250.0, 146950.0, 147550.0, 148450.0, 148450.0,
       148050.0, 148550.0, 148150.0, 147850.0, 148050.0, 147550.0,
       147650.0, 147050.0, 146750.0, 146350.0, 146850.0, 146950.0,
       146950.0, 146850.0, 146

In [18]:
Xtest_data.size()

torch.Size([7800, 7])

In [15]:
Ytest_data[:100]

tensor([1, 2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1, 1, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0,
        2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 1, 2, 2, 0, 2, 0, 2, 0, 2,
        0, 2, 1, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 2, 2, 2, 0, 2, 0, 0,
        0, 2, 0, 2, 2, 2, 1, 2, 2, 1, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 1, 0,
        2, 1, 2, 1])